In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [3]:
csv_folder_dir = "data/csv/out_of_sample/"
csv_list = [x for x in os.listdir(csv_folder_dir) if not x.startswith(".")]

prices_df = None

for csv_name in csv_list:
    csv_dir = os.path.join(csv_folder_dir, csv_name)
    current_df = pd.read_csv(csv_dir, 
                             usecols=['Date', 'Close'],
                             parse_dates=['Date'])
    current_df.rename(columns={'Close': f"{csv_name[:-4]}"}, inplace=True)
    
    # Merge the current DataFrame with the final DataFrame
    if prices_df is None:
        prices_df = current_df  # Initialize with the first DataFrame
    else:
        prices_df = pd.merge(prices_df, current_df, on='Date', how='outer')  # Merge on 'Date' 

# SP500

In [4]:
start = list(prices_df["Date"])[0]
end = list(prices_df["Date"])[-1]

In [5]:
import yfinance as yf

def get_stock_data(ticker, start_date, end_date=None):
    stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    stock_data.drop(["Open", "High", "Low", "Volume", 'Adj Close'], axis = 1, inplace = True)
    stock_data.reset_index(inplace = True, drop = True)
    return stock_data

sp500_price_df = get_stock_data("^GSPC", start, end)

In [6]:
sp500_price_df

,Close
0,3700.649902
1,3726.860107
2,3748.139893
3,3803.790039
4,3824.679932
...,...
971,5983.990234
972,5985.379883
973,5949.169922
974,5870.620117


In [7]:
window_size = 5
spx_long_returns = []
for today in range(5, len(prices_df) - 2 * window_size, 5):
    long_pos_start = sp500_price_df.iloc[today + window_size]
    long_pos_end = sp500_price_df.iloc[today + 2*window_size]
    long_pos_diff = (long_pos_end - long_pos_start)/long_pos_start
    long_portfolio_return = long_pos_diff.mean()
    spx_long_returns.append(long_portfolio_return)

In [8]:
spx_portfolio = pd.DataFrame({"Long portfolio": spx_long_returns})

In [9]:
spx_portfolio

,Long portfolio
0,0.013349
1,-0.006055
2,0.022194
3,0.005139
4,-0.001501
...,...
188,-0.001002
189,-0.005203
190,-0.019031
191,0.050530


In [20]:
spx_portfolio.mean()

Long portfolio    0.002539
dtype: float64

In [10]:
spx_portfolio.to_csv("data/spx_portfolio.csv", index = False)

In [18]:
spx_long_portfolio = 1000
spx_long_portfolio_history = []
for _, row in spx_portfolio.iterrows():
    spx_long_portfolio *= (1 + row["Long portfolio"]) 
    spx_long_portfolio_history.append(spx_long_portfolio)

In [19]:
spx_long_portfolio

1551.397704483871